In [51]:
import json
import urllib
import requests
import csv
import re
from datetime import datetime
from dateutil.tz import tzutc, tzlocal
import pytz

In [71]:
import pandas as pd
import calendar

colNames = ['Year', 'Month', 'Date', 'High', 'Low', 'Average', 'HDD', 'CDD', 'Rainfall']
#df = pd.read_json('/var/www/html/000/daily.txt')
df = pd.read_json('/Users/jameshayes/daily.txt')
df = df.drop(df.columns[[0, 10]], axis=1)

#
# Get month name from month number and make some conversions 
#

date = int(df['Date'])
month_num  = int(df['Month'])
month_name = calendar.month_name[month_num]
year = int(df['Year'])

high = int(df['High'])
low = int(df['Low'])
avg = int(df['Average'])
hdd = int(df['HDD'])
cdd = int(df['CDD'])
rain = float(df['Rainfall'])

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import pymysql as dbapi
import sys
import csv
from tabulate import tabulate
import sqlGet
import getDays
import logging

#
# Get the record high for the date
#

QUERY1 = """SELECT * FROM recHigh 
         WHERE Month = %s 
         AND Day = %s""" % (month_num, date)


db = dbapi.connect(host='3.135.162.69',user='chuckwx',passwd='jfr716!!00', database = 'trweather')

cur = db.cursor()
cur.execute(QUERY1)
result1 = cur.fetchall()
recordHigh = result1[0]
recHigh = int(recordHigh[1])
recHighYear = int(recordHigh[4])

QUERY2 = """SELECT * FROM recLow 
               WHERE Month = %s 
               AND Day = %s""" % (month_num, date)


db = dbapi.connect(host='3.135.162.69',user='chuckwx',passwd='jfr716!!00', database = 'trweather')

cur = db.cursor()
cur.execute(QUERY2)
result2 = cur.fetchall()
recYearNum =  len(result2)
recordLow = result2[0]
recLow = int(recordLow[1])
recLowYear = int(recordLow[4])

QUERY3 = """SELECT * FROM minOfMax 
         WHERE Month = %s 
         AND Day = %s""" % (month_num, date)


db = dbapi.connect(host='3.135.162.69',user='chuckwx',passwd='jfr716!!00', database = 'trweather')

cur = db.cursor()
cur.execute(QUERY3)
result3 = cur.fetchall()
recYearNum =  len(result3)
minOfMax = result3[0]
minHigh = int(minOfMax[1])
recminHighYear = int(minOfMax[4])

QUERY4 = """SELECT * FROM maxOfMin 
         WHERE Month = %s 
         AND Day = %s""" % (month_num, date)


db = dbapi.connect(host='3.135.162.69',user='chuckwx',passwd='jfr716!!00', database = 'trweather')

cur = db.cursor()
cur.execute(QUERY4)
result4 = cur.fetchall()
recYearNum =  len(result4)
maxOfMin = result4[0]
maxLow = int(maxOfMin[1])
recmaxLowYear = int(maxOfMin[4])

#
# Get the record rainfall for the date
#
    
QUERY5 = """SELECT * FROM recRain 
         WHERE Month = %s 
         AND Day = %s""" % (month_num, date)


db = dbapi.connect(host='3.135.162.69',user='chuckwx',passwd='jfr716!!00', database = 'trweather')

cur = db.cursor()
cur.execute(QUERY5)
result5 = cur.fetchall()
recYearNum =  len(result5)
recordRain = result5[0]
recRain = float(recordRain[1])
recRainYear = int(recordRain[4])


In [108]:
with open('/Users/jameshayes/dailyTest.html', 'w') as f:
    
    message = f'''
    <DOCTYPE html>
    <html>
    <link rel="stylesheet" type="text/css" href="/Users/jameshayes/forecastP.css" />
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Daily Almanac for Toms River, NJ</title>
    
    </head>
    <body>
    <div class="dailyTable">
    
    <p> Daily Almanac for Toms River, NJ<br>
        {month_name} {date}, {year}<br>    
    </p><br>
    
    <table border="2">
        <tr>
            <th>Daily Data</th>
            <th>Observed</th>   
            <th>Record High</th>
            <th>Record Low</th>  
        
        </tr>
        <tr>
            <td>Max Temperature</td><td>{high}</td><td>{recHigh} in {recHighYear}</td><td>{minHigh} in {recminHighYear}</td>
        </tr>
        <tr>
            <td>Min Temperature</td><td>{low}</td><td>{recLow} in {recLowYear}</td><td>{maxLow} in {recmaxLowYear}</td>
        </tr>
        <tr>
            <td>Avg Temperature</td><td>{avg}</td>
        </tr>
        <tr>
            <td>HDD</td><td>{hdd}</td>
        </tr>
        <tr>
            <td>CDD</td><td>{cdd}</td>            
        </tr>
        <tr>
            <td>Rainfall</td><td>{rain:.2f}</td><td>{recRain} in {recRainYear}</td>
        </tr>       
            
        
    </table>   
    </div>
   
    </body>
    </html>'''
    f.write(message)